In [1]:
import mysql.connector
import config
import time
import requests
import json

api_key = config.OMDB_api_key

#creates and opens a connection to the AWS database
def create_cnx_cur():
    cnx = mysql.connector.connect(
            host = config.host,
            user = config.user,
            passwd = config.password)

    cursor = cnx.cursor()
    return (cnx, cursor)


def close_connection(cnx, cursor):
    cursor.close()
    cnx.close()

#creates a new database in the AWS database
def create_database(cursor, database):
    try:
        #it will try to create a database with whatever name passed through
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
        #if this fails, the error will print out as a message
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        try:
            exit()
        except NameError:
            return

    #this is going to try the above function
    try:
        cursor.execute("USE {}".format(database))
    except mysql.connector.Error as err:
        print("Database {} does not exists.".format(database))
        if err.errno == errorcode.ER_BAD_DB_ERROR:
            create_database(cursor, database)
            print("Database {} created successfully.".format(database))
            cnx.database = database
        else:
            print(err)
            exit()


def add_tables_to_db(TABLES, cursor):
    from mysql.connector import errorcode
    for table_name in TABLES:
        table_description = TABLES[table_name]
        try:
            print("Creating table {}: ".format(table_name), end='')
            cursor.execute("""USE yelp""")
            cursor.execute(table_description)
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
                print("already exists.")
            else:
                print(err.msg)
        else:
            print("OK")

In [9]:
c = create_cnx_cur()

In [10]:
cnx = c[0]
cursor = c[1]

In [12]:
db_name = 'Moovies'

In [13]:
create_database(cursor, db_name)

In [ ]:
TABLES = {}

TABLES['movies'] = (
                        """CREATE TABLE movies (
                                 movieId VARCHAR(25),
                                 movieTitle TEXT,
                                 box-office REAL ,
                                 rating REAL,
                                  TEXT,
                                 PRIMARY KEY (businessId)
                                 ) ENGINE=InnoDB""")